In [1]:
from __future__ import print_function
import sys
import cv2
from random import randint
import numpy as np
 
trackerTypes = ['BOOSTING', 'MIL', 'KCF','TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
 
def createTrackerByName(trackerType):
    
  # Create a tracker based on tracker name
    if trackerType == trackerTypes[0]:
        tracker = cv2.TrackerBoosting_create()
  
    elif trackerType == trackerTypes[1]: 
        tracker = cv2.TrackerMIL_create()
   
    elif trackerType == trackerTypes[2]:
        tracker = cv2.TrackerKCF_create()
  
    elif trackerType == trackerTypes[3]:
        tracker = cv2.TrackerTLD_create()
   
    elif trackerType == trackerTypes[4]:
        tracker = cv2.TrackerMedianFlow_create()
    
    elif trackerType == trackerTypes[5]:
        tracker = cv2.TrackerGOTURN_create()
            
    elif trackerType == trackerTypes[6]:
        tracker = cv2.TrackerMOSSE_create()
        
    elif trackerType == trackerTypes[7]:
        tracker = cv2.TrackerCSRT_create()
        
    else:
        tracker = None
        print('Incorrect tracker name')
        print('Available trackers are:')
        for t in trackerTypes:
            print(t)
     
    return tracker


In [2]:
B = np.load("/home/siddhi/Desktop/RoadCrossingAssistant_FY_Project_Data/arrays_train/array4.npy", allow_pickle=True)

pq = B[50]
print(pq)

ab = [[b[0],b[1],b[2]-b[0],b[3]-b[1]] for b in pq]
print(ab)

[[531, 606, 685, 738], [944, 680, 1149, 885], [1631, 627, 1715, 714], [1315, 612, 1371, 675], [0, 484, 125, 635], [1389, 615, 1520, 714], [1520, 656, 1916, 1065]]
[[531, 606, 154, 132], [944, 680, 205, 205], [1631, 627, 84, 87], [1315, 612, 56, 63], [0, 484, 125, 151], [1389, 615, 131, 99], [1520, 656, 396, 409]]


In [3]:
videoPath = "/home/siddhi/Desktop/RoadCrossingAssistant_FY_Project_Data/videos_train/video4.MOV"
trackerType = "MOSSE"

def get_direction(videoPath, trackerType, boxes_list, frame_no, outputPath):
    
    k = 0 
    multiTracker_back = cv2.MultiTracker_create()
    multiTracker = cv2.MultiTracker_create()
    n = len(boxes_list)
    directions = [0]*n
    x1 = [0]*n
    x2 = [0]*n
    out = cv2.VideoWriter(outputPath,cv2.VideoWriter_fourcc('M','J','P','G'), 10, (1920,1080))
    cap = cv2.VideoCapture(videoPath)
    success, frame = cap.read()
    if not success:
        print('Failed to read video')
        sys.exit(1)

    while cap.isOpened() :
        success, frame = cap.read()
        if not success:
            break


        if( k == frame_no -3 ):
            frame_b3 = frame
        if ( k == frame_no - 2):
            frame_b2 = frame
        if( k == frame_no -1 ):
            frame_b1 = frame


        if k == frame_no:
            boxes_list = [tuple(l) for l in boxes_list]
            for i in range(n):
                box = boxes_list[i]
                x_temp = (box[0]+2*box[2])/2
                multiTracker.add(createTrackerByName(trackerType), frame, box)
                multiTracker_back.add(createTrackerByName(trackerType), frame, box)
                #x1[i] = x_temp
                #print('x1',i,x1[i])
            (success_back, boxes_back) = multiTracker_back.update(frame_b1)
            (success_back, boxes_back) = multiTracker_back.update(frame_b2)
            (success_back, boxes_back) = multiTracker_back.update(frame_b3)


            for i in range(n):
                    b = boxes_back[i]
                    x1[i] = (b[0]+2*b[2])/2
                    #print('x1',i,x1[i])

        (success, boxes) = multiTracker.update(frame)

        if(k == frame_no + 2):
            for i in range(n):
                b = boxes[i]
                x2[i] = (b[0]+2*b[2])/2
                #print('x2',i,x2[i])
                x_d =  x1[i] - x2[i]
                if x_d >= 3:
                    directions[i] = 1
                elif x_d <= -3:
                    directions[i] = -1

        # loop over the bounding boxes and draw then on the frame
        if(k>frame_no and k<frame_no+15):
            for i in range(n):
                box = boxes[i]
                (x,y,w,h) = [int(v) for v in box]
                if(directions[i] == 0):
                    cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)
                elif(directions[i] == 1):
                    cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
                elif(directions[i] == -1):
                    cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255), 2)

        cv2.imshow('frame', frame)
        key = cv2.waitKey(1) & 0xFF
            
        if key == ord("q"):
            break
        out.write(frame)
        #print(k, directions)
        k = k+1
    
    out.release()
    cap.release()
    cv2.destroyAllWindows()
    return 0
        
x = get_direction(videoPath, trackerType, ab, 50, '/home/siddhi/Desktop/RoadCrossingAssistant_FY_Project_Data/outpy.avi')